In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.sql.functions import col
import os
from pyspark.sql.functions import array_join, col

def create_spark_session():
    spark_master_url = os.getenv("SPARK_MASTER_URL", "spark://localhost:7077")
    return (
        SparkSession.builder.appName("ReviewClassification")
        .config("spark.master", "local[1]")
        .getOrCreate()
    )

In [2]:
def load_data_from_csv(spark, file_path):
    return spark.read.option("multiline", "true").option("escape", "\"").csv(file_path, header=True, inferSchema=True, sep=",")

df = load_data_from_csv(create_spark_session(), "spark/data/vectorized_reviews.csv")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/26 18:08:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df

DataFrame[_id: string, username: string, faculty: string, year: int, opinion_weight: int, date: date, professor: string, rating: string, vote_rate: int, course: string, review: string, post_url: string, language: string, vectors: string]

In [4]:
#show language column from the dataframe
df.show(5)

+--------------------+---------+-------------+----+--------------+----------+------------------+------+---------+---------+--------------------+--------------------+--------+--------------------+
|                 _id| username|      faculty|year|opinion_weight|      date|         professor|rating|vote_rate|   course|              review|            post_url|language|             vectors|
+--------------------+---------+-------------+----+--------------+----------+------------------+------+---------+---------+--------------------+--------------------+--------+--------------------+
|680aab9873fce4fd2...|Anonymous|         NULL|NULL|             2|2013-11-24|Elżbieta Marszałek|     5|        0|     NULL|Funky  \n \n\n \n...|https://polwro.co...|      pl|0.277272760868072...|
|680aab9873fce4fd2...|Anonymous|       W-8 IZ|   1|          NULL|2014-03-19|Elżbieta Marszałek|     5|        0|     NULL|Kurs:Body ball  \...|https://polwro.co...|      pl|-0.05305481702089...|
|680aab98dfd3e08e4..

In [5]:
# show only vectors
df.select("vectors").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------